In [1]:
import pandas as pd
import numpy as np

In [2]:
#Load the u.user file into a dataframe
u_cols = ['user_id', 'age', 'sex', 'occupation', 'zip_code']
users = pd.read_csv('C:/RecoSys/Data/u.user', sep='|', names=u_cols, encoding='latin-1')
users.head()

,user_id,age,sex,occupation,zip_code
0,1,24,M,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067
3,4,24,M,technician,43537
4,5,33,F,other,15213


In [3]:
#Load the u.items file into a dataframe
i_cols = ['movie_id', 'title', 'release date', 'video release date', 'IMDB URL', 'unknown', 
          'Action', 'Adventure', 'Animation', 'Children\'s', 'Comedy', 'Crime', 'Documentary', 
          'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 
          'Thriller', 'War', 'Western']
movies = pd.read_csv('C:/RecoSys/Data/u.item', sep='|', names=i_cols, encoding='latin-1')
movies.head()

,movie_id,title,release date,video release date,IMDB URL,unknown,Action,Adventure,Animation,Children's,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [4]:
#Remove all information except movie ID and title
movies = movies[['movie_id', 'title']]

In [5]:
#Load the u.data file into a dataframe
r_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv('C:/RecoSys/Data/u.data', sep='\t', names=r_cols, encoding='latin-1')
ratings.head()

,user_id,movie_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [6]:
#Drop the timestamp column
ratings = ratings.drop('timestamp', axis=1)

In [7]:
#Import the train_test_split function
from sklearn.model_selection import train_test_split

#Assign x as the original ratings dataframe and y as the user_id column of ratings
x = ratings.copy()
y = ratings['user_id']

#Split into training and test datasets, stratified along user_id
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, stratify=y)

In [8]:
#Import the mean_squared_error function
from sklearn.metrics import mean_squared_error

In [9]:
#Function that computes the root mean squared error (or RMSE)
def RMSE(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

In [10]:
#Define the baseline model 
def baseline(user_id, movie_id):
    try:
        rating = train_mean[movie_id]
    except:
        rating = 3.0
    return rating

In [11]:
#Function to compute the RMSE score obtained on the testing set by a model
def score(cf_model):
    #Construct a list of user-movie tuples from the testing dataset
    id_pairs = zip(x_test['user_id'], x_test['movie_id'])
    #Predict the rating for every user-movie tuple
    y_pred = np.array([cf_model(user, movie) for (user, movie) in id_pairs])
    #Extract the actual ratings given by the users in the test data
    y_true = np.array(x_test['rating'])
    #Return the final RMSE score
    return RMSE(y_true, y_pred)

In [12]:
#Build the ratings matrix using pivot_table function
r_matrix = x_train.pivot_table(values='rating', index='user_id', columns='movie_id')
r_matrix.head()

movie_id,1,2,3,4,5,6,7,8,9,10,...,1671,1672,1673,1674,1675,1676,1677,1678,1679,1681
user_id,,,,,,,,,,,,,,,,,,,,,
1,5.0,3.0,4.0,3.0,3.0,NaN,4.0,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
#Merge the original users dataframe with the training set
merged_df = pd.merge(x_train, users)

In [14]:
merged_df.head()

,user_id,movie_id,rating,age,sex,occupation,zip_code
0,577,95,5,36,F,student,77845
1,577,176,5,36,F,student,77845
2,577,313,4,36,F,student,77845
3,577,240,3,36,F,student,77845
4,577,98,4,36,F,student,77845


### 사용자 나이 기준 demographic 기반 추천

### 1. ~19, 20~29, 30~39, 40~49, 50~59, 60~ 

In [15]:
merged_copy1 = merged_df.copy()

In [16]:
def age_grouping1(x):
    if x <= 19:
        return 1
    elif x <= 29:
        return 2
    elif x <= 39:
        return 3
    elif x <= 49:
        return 4
    elif x <= 59:
        return 5
    else:
        return 6

In [17]:
merged_copy1 ['age'] = merged_copy1 ['age'].apply(age_grouping1)

In [18]:
#Compute the mean rating of every movie by gender
age_mean1 = merged_copy1 [['movie_id', 'age', 'rating']].groupby(['movie_id', 'age'])['rating'].mean()

In [19]:
#Set the index of the users dataframe to the user_id
users = users.set_index('user_id')

In [20]:
users_copy1 = users.copy()

In [21]:
users_copy1['age'] = users_copy1['age'].apply(age_grouping1)

In [22]:
#Age Based Collaborative Filter using Mean Ratings
def cf_age1(user_id, movie_id):
    #Check if movie_id exists in r_matrix
    if movie_id in r_matrix:
        #Identify the age of the user
        age = users_copy1.loc[user_id]['age']
        
        #Check if the age has rated the movie
        if age in age_mean1[movie_id]:
            #Compute the mearn rating given by that age to the movie
            age_rating = age_mean1[movie_id][age]
        else:
            age_rating = 3.0
    else:
        #Default to a rating of 3.0 in the absence of any information
        age_rating = 3.0
    return age_rating

In [23]:
score(cf_age1)

1.0632303872838096

### 2. ~25, 26~39, 40~55, 56~

In [24]:
merged_copy2 = merged_df.copy()

In [25]:
def age_grouping2(x):
    if x <= 25:
        return 1
    elif x <= 39:
        return 2
    elif x <= 55:
        return 3
    else:
        return 4

In [26]:
merged_copy2 ['age'] = merged_copy2 ['age'].apply(age_grouping2)

In [27]:
#Compute the mean rating of every movie by gender
age_mean2 = merged_copy2 [['movie_id', 'age', 'rating']].groupby(['movie_id', 'age'])['rating'].mean()

In [28]:
users_copy2 = users.copy()

In [29]:
users_copy2['age'] = users_copy2['age'].apply(age_grouping2)

In [30]:
#Age Based Collaborative Filter using Mean Ratings
def cf_age2(user_id, movie_id):
    #Check if movie_id exists in r_matrix
    if movie_id in r_matrix:
        #Identify the age of the user
        age = users_copy2.loc[user_id]['age']
        
        #Check if the age has rated the movie
        if age in age_mean2[movie_id]:
            #Compute the mearn rating given by that age to the movie
            age_rating = age_mean2[movie_id][age]
        else:
            age_rating = 3.0
    else:
        #Default to a rating of 3.0 in the absence of any information
        age_rating = 3.0
    return age_rating

In [31]:
score(cf_age2)

1.0489866196388977

### 3. ~29, 30~49, 50~

In [32]:
merged_copy3 = merged_df.copy()

In [33]:
def age_grouping3(x):
    if x <= 29:
        return 1
    elif x <= 49:
        return 2
    else:
        return 3

In [34]:
merged_copy3 ['age'] = merged_copy3 ['age'].apply(age_grouping3)

In [35]:
#Compute the mean rating of every movie by gender
age_mean3 = merged_copy3 [['movie_id', 'age', 'rating']].groupby(['movie_id', 'age'])['rating'].mean()

In [36]:
users_copy3 = users.copy()

In [37]:
users_copy3['age'] = users_copy3['age'].apply(age_grouping3)

In [38]:
#Age Based Collaborative Filter using Mean Ratings
def cf_age3(user_id, movie_id):
    #Check if movie_id exists in r_matrix
    if movie_id in r_matrix:
        #Identify the age of the user
        age = users_copy3.loc[user_id]['age']
        
        #Check if the age has rated the movie
        if age in age_mean3[movie_id]:
            #Compute the mearn rating given by that age to the movie
            age_rating = age_mean3[movie_id][age]
        else:
            age_rating = 3.0
    else:
        #Default to a rating of 3.0 in the absence of any information
        age_rating = 3.0
    return age_rating

In [39]:
score(cf_age3)

1.0400887937365229

### 결과 한번에 비교하기 -> 기준 3의 결과가 가장 좋음

In [40]:
print("기준1 RMSE:" , score(cf_age1))
print("기준2 RMSE:" , score(cf_age2))
print("기준3 RMSE:" , score(cf_age3))

기준1 RMSE: 1.0632303872838096
기준2 RMSE: 1.0489866196388977
기준3 RMSE: 1.0400887937365229
